# THINNER - FITTER - WISER - HEALTHIER

# CREATING FOOD DATABASE

We will start building our dataset scrapping the information from [BEDCA](https://www.bedca.net/bdpub/index.php) the Spanish Food Composition Database published by the Ministry of Science and Innovation and under the coordination and funding of the Spanish Agency for Food Safety and Nutrition of the Ministry of Health, Social Services and Equality.

The food composition values collected in this database have been obtained from different sources that include laboratories, the food industry and scientific publications or have been calculated by the agency.

Our databse will be stored in a `pandas.DataFrame`.

### PREREQUISITES

In [ ]:
#!pip install selenium
#!pip install webdriver-manager

### IMPORTS

In [ ]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait  # WebDriverWait(driver, 5) # wait for the table list to load # table = WebDriverWait(driver, 10)
from webdriver_manager.firefox import GeckoDriverManager

### BUILDING THE DRIVER

In [ ]:
srv = Service(GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 95.0
Get LATEST geckodriver version for 95.0 firefox
Driver [/Users/mabatalla/.wdm/drivers/geckodriver/macos/v0.30.0/geckodriver] found in cache


In [ ]:
driver = webdriver.Firefox(service=srv)
driver.maximize_window()

### NAVIGATING TO THE DATA

In [ ]:
# Access website
url = 'https://www.bedca.net/bdpub/index.php'
driver.get(url)
WebDriverWait(driver, 5)

# Access database
driver.find_element(By.ID, 'navigation').find_element(By.LINK_TEXT, 'Consulta').click()
WebDriverWait(driver, 5)

# Show items alphabetically ordered
driver.find_element(By.ID, 'Alfabetica').click()
WebDriverWait(driver, 5)

# Access all items
driver.find_element(By.LINK_TEXT, 'Todos').click()

### ACCESSING A SINGLE ITEM INFO

In [ ]:
# The table is made of row-a and row-b classes, let's put them into a list
get_row_a = driver.find_elements(By.CLASS_NAME, 'row-a')
get_row_b = driver.find_elements(By.CLASS_NAME, 'row-b')

In [ ]:
# Now let's unify both lists
foodindex = zip(get_row_a, get_row_b)

In [ ]:
# Checkpoint - The dataset has 968 entries

print(len(get_row_a) + len(get_row_b))

entries = sum(2 for i in foodindex)  # This zip object is made of 2-elements tuples
print(entries)                       # one element from get_row_a and one from get_row_b
                                     # that's why I add 2 for each element in foodindex

84
84


In [ ]:
# Access one item ('Arroz con leche')
# To find item index
foodname = []

for product_a, product_b in foodindex:
    foodname.append(product_a.find_element(By.ID, 'foodname').text)
    foodname.append(product_b.find_element(By.ID, 'foodname').text)

In [ ]:
foodname.index('Arroz con leche')

65

In [ ]:
# Index 65 in foodindex matches with 32 index in row-b (65 // 2)
# 0,  1,  2,  3,  4,  5  ...
# a0, b0, a1, b1, a2, b2 ...

# Checkpoint - Accesing the correct product
print(driver.find_elements(By.CLASS_NAME, 'row-b')[32].find_element(By.ID, 'foodname').find_element(By.TAG_NAME, 'a').text)

Arroz con leche


In [ ]:
get_row_b[32].find_element(By.TAG_NAME, 'a').click()

Now, lets take a look to all available info:

[IMG]

The relevant info for us is inside id='content2':
* Go back
___
* Basic info
    * Foodname (ESP)
    * Foodname (ENG)
    * Quantity measured
___
* Macros:
    * Energy, Carbs, Prot, Fats
* Complementary:
    * Water (humidity), Fiber, Saturated fats, Unsaturated fats
* Flags:
    * Palmitic acid (present), Salt (>1), Energy (>275), Cabrs (>10)
* Minerals:
    * Calcium, iron, potassium, magnesium, sodium, phosphorus, iodide, selenium, zinc
* Vitamnins:
    * A, B1 (Thiamine), B2 (riboflavin), B3 (niacin), B6, B9 (folic acid), B12, C (ascorbic acid), D, E

In [ ]:
# Checking texts found
h4 = driver.find_element(By.ID, 'content2').find_elements(By.CSS_SELECTOR, 'h4')
p = driver.find_element(By.ID, 'content2').find_elements(By.CSS_SELECTOR, 'p')
span = driver.find_element(By.ID, 'content2').find_elements(By.TAG_NAME, 'span')

for text in range(len(h4)): print(h4[text].text)

for text in range(len(p)): print(p[text].text)

for text in range(len(span)): print(span[text].text)

Arroz con leche
Distribución de la Energía total
Información de composición (por 100 g de porción comestible)
Listado anterior

English name: Rice with milk
Nombre científico:
Código foodex: A041E - A03VA (Composite dishes) - A03VB (Dishes, incl. Ready to eat meals (excluding soups and salads))
Parte comestible (%): 100
Descripción:
Listado anterior
A03VA
A03VB (Dishes, incl. Ready to eat meals (excluding soups and salads))


In [12]:
# Checking tables found
table = driver.find_element(By.ID, 'content2').find_elements(By.CSS_SELECTOR, 'table')

print(table[0].text)

print(table[1].text)

print(table[2].text)

English name: Rice with milk
Nombre científico:
Código foodex: A041E - A03VA (Composite dishes) - A03VB (Dishes, incl. Ready to eat meals (excluding soups and salads))
Parte comestible (%): 100
Descripción:

Componente Valor Unidad Fuente
Proximales
alcohol (etanol) 0 g 38
energía, total 383 (92) kJ (kcal) 236
grasa, total (lipidos totales) 1.3 g 38
proteina, total 3.3 g 38
agua (humedad) 78.9 g 38
Hidratos de Carbono
fibra, dietetica total 0.1 g 38
carbohidratos 16.4 g 38
Grasas
ácido graso 22:6 n-3 (ácido docosahexaenóico) - - -
ácidos grasos, monoinsaturados totales 0.31 g 38
ácidos grasos, poliinsaturados totales 0.08 g 38
ácidos grasos saturados totales 0.77 g 38
ácido graso 12:0 (láurico) - - -
ácido graso 14:0 (ácido mirístico) - - -
ácido graso 16:0 (ácido palmítico) - - -
ácido graso 18:0 (ácido esteárico) - - -
ácido graso 18:1 n-9 cis (ácido oléico) - - -
colesterol 9 mg 38
ácido graso 18:2 - - -
ácido graso 18:3 - - -
ácido graso 20:4 n-6 (ácido araquidónico) - - -
ácido gr

Lets get the relevant elements

In [21]:
# Go back
back = driver.find_element(By.ID, 'Todos')

back.click()

Lets get the relevant info

In [113]:
# General info

foodname_ESP = driver.find_element(By.ID, 'content2').find_elements(By.CSS_SELECTOR, 'h4')[0].text
foodname_ENG = driver.find_element(By.ID, 'content2').find_elements(By.CSS_SELECTOR, 'p')[2].text[14:]
quantity = driver.find_element(By.ID, 'content2').find_elements(By.CSS_SELECTOR, 'h4')[2].text[32:][:-25]


general_info = {'foodname_ESP': foodname_ESP, 'foodname_ENG': foodname_ENG, 'quantity': float(quantity)}

general_info

{'foodname_ESP': 'Arroz con leche',
 'foodname_ENG': 'Rice with milk',
 'quantity': 100.0}

In [14]:
# Table items
nutritional_facts = table[2].text.splitlines()

nutritional_facts

['Componente Valor Unidad Fuente',
 'Proximales',
 'alcohol (etanol) 0 g 38',
 'energía, total 383 (92) kJ (kcal) 236',
 'grasa, total (lipidos totales) 1.3 g 38',
 'proteina, total 3.3 g 38',
 'agua (humedad) 78.9 g 38',
 'Hidratos de Carbono',
 'fibra, dietetica total 0.1 g 38',
 'carbohidratos 16.4 g 38',
 'Grasas',
 'ácido graso 22:6 n-3 (ácido docosahexaenóico) - - -',
 'ácidos grasos, monoinsaturados totales 0.31 g 38',
 'ácidos grasos, poliinsaturados totales 0.08 g 38',
 'ácidos grasos saturados totales 0.77 g 38',
 'ácido graso 12:0 (láurico) - - -',
 'ácido graso 14:0 (ácido mirístico) - - -',
 'ácido graso 16:0 (ácido palmítico) - - -',
 'ácido graso 18:0 (ácido esteárico) - - -',
 'ácido graso 18:1 n-9 cis (ácido oléico) - - -',
 'colesterol 9 mg 38',
 'ácido graso 18:2 - - -',
 'ácido graso 18:3 - - -',
 'ácido graso 20:4 n-6 (ácido araquidónico) - - -',
 'ácido graso 20:5 (ácido eicosapentaenóico) - - -',
 'Vitaminas',
 'Vitamina A equivalentes de retinol de actividades d

In [15]:
# Get relevant facts indexes
relevant_facts = [#'Componente Valor Unidad Fuente',
                  #'Proximales',
                  #'alcohol (etanol) 0 g 38',
                  'energía, total 383 (92) kJ (kcal) 236',
                  'grasa, total (lipidos totales) 1.3 g 38',
                  'proteina, total 3.3 g 38',
                  'agua (humedad) 78.9 g 38',
                  #'Hidratos de Carbono',
                  'fibra, dietetica total 0.1 g 38',
                  'carbohidratos 16.4 g 38',
                  #'Grasas',
                  #'ácido graso 22:6 n-3 (ácido docosahexaenóico) - - -',
                  'ácidos grasos, monoinsaturados totales 0.31 g 38',
                  'ácidos grasos, poliinsaturados totales 0.08 g 38',
                  'ácidos grasos saturados totales 0.77 g 38',
                  #'ácido graso 12:0 (láurico) - - -',
                  #'ácido graso 14:0 (ácido mirístico) - - -',
                  'ácido graso 16:0 (ácido palmítico) - - -',
                  #'ácido graso 18:0 (ácido esteárico) - - -',
                  #'ácido graso 18:1 n-9 cis (ácido oléico) - - -',
                  #'colesterol 9 mg 38',
                  #'ácido graso 18:2 - - -',
                  #'ácido graso 18:3 - - -',
                  #'ácido graso 20:4 n-6 (ácido araquidónico) - - -',
                  #'ácido graso 20:5 (ácido eicosapentaenóico) - - -',
                  #'Vitaminas',
                  'Vitamina A equivalentes de retinol de actividades de retinos y carotenoides 18 ug 38',
                  'Vitamina D 0.1 ug 38',
                  'Viamina E equivalentes de alfa tocoferol de actividades de vitámeros E 0.15 mg 38',
                  'folato, total 5 ug 38',
                  'equivalentes de niacina, totales 0.81 mg 38',
                  'riboflavina 0.13 mg 38',
                  'tiamina 0.04 mg 38',
                  'Vitamina B-12 0.24 ug 38',
                  'Vitamina B-6, Total 0.09 mg 38',
                  'Vitamina C (ácido ascórbico) 0 mg 38',
                  #'Minerales',
                  'calcio 109 mg 38',
                  'hierro, total 0.2 mg 38',
                  'potasio 138 mg 38',
                  'magnesio 14 mg 38',
                  'sodio 38 mg 38',
                  'fósforo 96 mg 38',
                  'ioduro 6.3 ug 38',
                  'selenio, total 1.9 ug 38',
                  'zinc (cinc) 0.3 mg 38']

relevant_facts_ind = []

for fact in relevant_facts:
    relevant_facts_ind.append(int(nutritional_facts.index(fact)))

In [16]:
# Checkpoint - Show each fact with its corresponding index
for i in relevant_facts_ind:
    print(i, nutritional_facts[i], sep=' - ')

3 - energía, total 383 (92) kJ (kcal) 236
4 - grasa, total (lipidos totales) 1.3 g 38
5 - proteina, total 3.3 g 38
6 - agua (humedad) 78.9 g 38
8 - fibra, dietetica total 0.1 g 38
9 - carbohidratos 16.4 g 38
12 - ácidos grasos, monoinsaturados totales 0.31 g 38
13 - ácidos grasos, poliinsaturados totales 0.08 g 38
14 - ácidos grasos saturados totales 0.77 g 38
17 - ácido graso 16:0 (ácido palmítico) - - -
26 - Vitamina A equivalentes de retinol de actividades de retinos y carotenoides 18 ug 38
27 - Vitamina D 0.1 ug 38
28 - Viamina E equivalentes de alfa tocoferol de actividades de vitámeros E 0.15 mg 38
29 - folato, total 5 ug 38
30 - equivalentes de niacina, totales 0.81 mg 38
31 - riboflavina 0.13 mg 38
32 - tiamina 0.04 mg 38
33 - Vitamina B-12 0.24 ug 38
34 - Vitamina B-6, Total 0.09 mg 38
35 - Vitamina C (ácido ascórbico) 0 mg 38
37 - calcio 109 mg 38
38 - hierro, total 0.2 mg 38
39 - potasio 138 mg 38
40 - magnesio 14 mg 38
41 - sodio 38 mg 38
42 - fósforo 96 mg 38
43 - ioduro 6

In [90]:
# Rename the relevant facts dropping non relevant
relevant_facts = ['energy',
                  'fats',
                  'prot',
                  'water',
                  'fiber',
                  'carbs',
                  'm_insat_fats',
                  'p_insat_fats',
                  'sat_fats',
                  'palm_acid',
                  'A',
                  'D',
                  'E',
                  'B9',
                  'B3',
                  'B2',
                  'B1',
                  'B12',
                  'B6',
                  'C',
                  'calcium',
                  'iron',
                  'potassium',
                  'magnesium',
                  'sodium',
                  'phosphorus',
                  'iodide',
                  'selenium',
                  'zinc']

In [ ]:
# Create a dict with facts as keys and index as values
facts = dict(zip(relevant_facts, relevant_facts_ind))

# Substitute indexes for scraped value
# Use 'composite' as value in case there is an error, this
# values need further transformation
for fact in facts:
    try:
        facts[fact] = float(nutritional_facts[facts.get(fact)].split()[-3])
    except:
        facts[fact] = 'composite'

facts

{'energy': 'composite',
 'fats': 1.3,
 'prot': 3.3,
 'water': 78.9,
 'fiber': 0.1,
 'carbs': 16.4,
 'm_insat_fats': 0.31,
 'p_insat_fats': 0.08,
 'sat_fats': 0.77,
 'palm_acid': 'composite',
 'A': 18.0,
 'D': 0.1,
 'E': 0.15,
 'B9': 5.0,
 'B3': 0.81,
 'B2': 0.13,
 'B1': 0.04,
 'B12': 0.24,
 'B6': 0.09,
 'C': 0.0,
 'calcium': 109.0,
 'iron': 0.2,
 'potassium': 138.0,
 'magnesium': 14.0,
 'sodium': 38.0,
 'phosphorus': 96.0,
 'iodide': 6.3,
 'selenium': 1.9,
 'zinc': 0.3}

In [101]:
# Transformation of some composite facts
facts['energy'] = float(nutritional_facts[3].split(' ')[3][1:-1])

facts['unsat_fats'] = float(nutritional_facts[12].split()[-3]) + float(nutritional_facts[13].split()[-3])
facts.pop('m_insat_fats', 'p_insat_fats')

palm_acid = nutritional_facts[17].split()[-3]
if palm_acid == '-':
    facts['palm_acid'] = False
else:
    facts['palm_acid'] = True

# Generating other composite facts that will be used later
facts['salt'] = 'composite'
facts['high_energy'] = 'composite'
facts['high_carbs'] = 'composite'
facts['high_fats'] = 'composite'
facts['quantity'] = quantity
    
facts

{'energy': 92.0,
 'fats': 1.3,
 'prot': 3.3,
 'water': 78.9,
 'fiber': 0.1,
 'carbs': 16.4,
 'p_insat_fats': 0.08,
 'sat_fats': 0.77,
 'palm_acid': False,
 'A': 18.0,
 'D': 0.1,
 'E': 0.15,
 'B9': 5.0,
 'B3': 0.81,
 'B2': 0.13,
 'B1': 0.04,
 'B12': 0.24,
 'B6': 0.09,
 'C': 0.0,
 'calcium': 109.0,
 'iron': 0.2,
 'potassium': 138.0,
 'magnesium': 14.0,
 'sodium': 38.0,
 'phosphorus': 96.0,
 'iodide': 6.3,
 'selenium': 1.9,
 'zinc': 0.3,
 'unsat_fats': 0.39,
 'salt': 'composite',
 'high_energy': 'composite',
 'high_carbs': 'composite',
 'high_fats': 'composite',
 'quantity': '100'}

In [125]:
item = pd.Series({**general_info, **facts})

item

foodname_ESP    Arroz con leche
foodname_ENG     Rice with milk
quantity                    100
energy                     92.0
fats                        1.3
prot                        3.3
water                      78.9
fiber                       0.1
carbs                      16.4
p_insat_fats               0.08
sat_fats                   0.77
palm_acid                 False
A                          18.0
D                           0.1
E                          0.15
B9                          5.0
B3                         0.81
B2                         0.13
B1                         0.04
B12                        0.24
B6                         0.09
C                           0.0
calcium                   109.0
iron                        0.2
potassium                 138.0
magnesium                  14.0
sodium                     38.0
phosphorus                 96.0
iodide                      6.3
selenium                    1.9
zinc                        0.3
unsat_fa

In [147]:
fut = pd.DataFrame(item)
#fut = fut.T

fut

,0
foodname_ESP,Arroz con leche
foodname_ENG,Rice with milk
quantity,100
energy,92.0
fats,1.3
prot,3.3
water,78.9
fiber,0.1
carbs,16.4
p_insat_fats,0.08


In [149]:
fut.set_index(['foodname_ESP', 'foodname_ENG'], inplace=True)
#fut.T

KeyError: "None of ['foodname_ESP', 'foodname_ENG'] are in the columns"

In [156]:
fut['pera'] = item

fut.T

,foodname_ESP,foodname_ENG,quantity,energy,fats,prot,water,fiber,carbs,p_insat_fats,...,sodium,phosphorus,iodide,selenium,zinc,unsat_fats,salt,high_energy,high_carbs,high_fats
0,Arroz con leche,Rice with milk,100,92.0,1.3,3.3,78.9,0.1,16.4,0.08,...,38.0,96.0,6.3,1.9,0.3,0.39,composite,composite,composite,composite
pera,Arroz con leche,Rice with milk,100,92.0,1.3,3.3,78.9,0.1,16.4,0.08,...,38.0,96.0,6.3,1.9,0.3,0.39,composite,composite,composite,composite
8,Arroz con leche,Rice with milk,100,92.0,1.3,3.3,78.9,0.1,16.4,0.08,...,38.0,96.0,6.3,1.9,0.3,0.39,composite,composite,composite,composite


### BUILDING THE DATAFRAME

In [107]:
macros = ['energy', 'fats', 'prot', 'carbs']
df[macros]

,,energy,fats,prot,carbs
Foodname_ESP,Foodname_ENG,,,,
Arroz con leche,Rice with milk,92.0,1.3,3.3,16.4


In [ ]:
complementary = ['water', 'fiber', 'sat_fats', 'unsat_fats']
df[complementary]

flags = ['palm_acid', 'salt', 'high_energy', 'high_carbs', 'high_fats']
df[flags]

minerals = ['calcium', 'iron', 'potassium', 'magnesium', 'sodium', 'phosphorus', 'iodide', 'selenium', 'zinc']
df[minerals]

vitamins = ['A', 'D', 'E', 'B9', 'B3', 'B2', 'B1', 'B12', 'B6', 'C']
df[vitamins]